In [1]:
import argparse
import os
import pandas as pd
import pickle
import tensorflow as tf
import scipy.io as sc
import numpy as np
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from xuejie import *


In [2]:
import argparse
import os
import pandas as pd
import pickle
import tensorflow as tf
import scipy.io as sc
import numpy as np
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from xuejie import *


def one_hot(y_):
    '''
    # this function is used to transfer one column label to one hot label
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    @return: one_hot_code
    '''
    y_ = y_.reshape(len(y_))
    y_ = y_.astype(int)
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]


def butter_bandpass(lowcut, highcut, fs, order=5):
    '''
    extract the delta from the data
    @return: b,a
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    '''
    @return: y
    '''
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def data_1Dto2D(data, Y=10, X=11):

    '''

        :param data: change the 1D data to 2D data
        :param Y: the length of 2D
        :param X:  the width of 2D
        :return: 2D data
        '''
    data_2D = np.zeros([Y, X])
    data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,    data[0],  data[1],  data[2], 	      0,  	     0, 	   0, 	 	 0)
    data_2D[1] = (	  	 0, 	   0,  	   	 0,         0,   data[3],        0,  data[4],         0, 	   	 0,   	   0, 	 	 0)
    data_2D[2] = (	  	 0, data[5],    data[6],  data[7],   data[8],  data[9], data[10], data[11],  data[12],  data[13], 	 	 0)
    data_2D[3] = (	  	 0, data[14],  data[15],  data[16],  data[17], data[18], data[19], data[20],  data[21],  data[22], 		 0)
    data_2D[4] = (       0, data[23],  data[24],  data[25],  data[26], data[27], data[28], data[29],  data[30],  data[31],       0)
    data_2D[5] = (	  	 0, data[32],  data[33],  data[34],  data[35], data[36], data[37], data[38],  data[39],  data[40], 		 0)
    data_2D[6] = (	  	 0, data[41],  data[42],  data[43],  data[44], data[45], data[46], data[47],  data[48],  data[49], 		 0)
    data_2D[7] = (	  	 0, 	   0,  data[50],  data[51],  data[52], data[53], data[54], data[55],  data[56], 	   0, 		 0)
    data_2D[8] = (	  	 0, 	   0, 	 	 0,   data[57],  data[58], data[59], data[60], data[61], 	   	 0, 	   0, 		 0)
    data_2D[9] = (	  	 0, 	   0, 	 	 0, 	     0, 	    0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0)
    #return shape:10*11
    return data_2D

def norm_dataset(dataset_1D):
    norm_dataset_1D = np.zeros([dataset_1D.shape[0], 62])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_1D[i] = feature_normalize(dataset_1D[i])
    # return shape: m*64
    return norm_dataset_1D

def feature_normalize(data):
    mean = data[data.nonzero()].mean()
    sigma = data[data.nonzero()].std()
    data_normalized = data
    data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
    # return shape: 10*11
    return data_normalized

def dataset_1Dto2D(dataset_1D):
    dataset_2D = np.zeros([dataset_1D.shape[0], 10, 11])
    for i in range(dataset_1D.shape[0]):
         dataset_2D[i] = data_1Dto2D(dataset_1D[i])
    # return shape: m*10*11
    return dataset_2D

def norm_dataset_1Dto2D(dataset_1D):
    norm_dataset_2D = np.zeros([dataset_1D.shape[0], 10, 11])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_2D[i] = feature_normalize(data_1Dto2D(dataset_1D[i]))
    # return shape: m*10*11
    return norm_dataset_2D

def windows(data, size):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += (size)

def segment_signal_without_transition(data, label, window_size):
    for (start, end) in windows(data, window_size):
        # print(data.shape)
        if((len(data[start:end]) == window_size) and (len(set(label))==1)):
            if(start == 0):
                segments = data[start:end]
                # labels = stats.mode(label[start:end])[0][0]
                labels_one = label[0]

            else:
                segments = np.vstack([segments, data[start:end]])
                labels_one = np.append(labels_one, label[0])
                # labels = np.append(labels, stats.mode(label[start:end])[0][0])

    print("the shape of segments is:", segments.shape)
    print("the shape of labels_one is:", labels_one.shape)
    return segments, labels_one

def get_a_student_eegs(student_data_path):
    '''
    @param mat_path:
    @return student_name:
    @return eegs:
    '''
    import scipy.io
    student_name = os.path.basename(student_data_path).replace('.mat', '')
    a_student_data = scipy.io.loadmat(student_data_path)
    eeg_keys = [i for i in a_student_data.keys() if i[0] != '_']
    #print(eeg_keys)
    eegs = []
    for eeg_key in eeg_keys:
        if get_label(eeg_key) in {3, 4, 7, 12, 15}: ##positive
            eegs.append(a_student_data[eeg_key])
    return eegs
# if get_label(eeg_key) in {1, 6, 9, 10, 14}:   ##positive
# if get_label(eeg_key) in {3, 4, 7, 12, 15}:  #negative emotion
# if get_label(eeg_key) in {2, 5, 8, 11, 13}:  #middle emotion

In [3]:
students_data_dir = './cat'
student_data_paths = get_student_data_paths('./cat')

labels = []
eg = []
i = 0
for student_data_path in student_data_paths:
	num = ''.join([x for x in student_data_path if x.isdigit()])
	temp = int(num)-1
	label = temp * np.ones((5))
	eegs = get_a_student_eegs(student_data_path = os.path.join(students_data_dir, student_data_path))
	labels.append(label)
	eg.append(eegs)

label_all = []

fs = 200
lowcut = 4
highcut = 40
# 四种波的频率范围


for i in range(len(eg)):
	for j in range(len(eg[i])):
		s = np.array(eg[i][j]) #s shape (62,47001)
		for q in range(62):
			print(s[q])
			s[q] = butter_bandpass_filter(s[q], lowcut, highcut, fs, order=5)
			print(s[q])
		eg[i][j] = s


#print('len(eg)', len(eg))


for i in range(len(eg)):
	for j in range(len(eg[i])):
		data_f1 = np.array(eg[i][j])
		data_f1 = np.transpose(data_f1)

		#x_all = preprocessing.scale(data_f1)
		x_all = norm_dataset(data_f1)
		#x_all = data_f1
		window_size = 200*5
		label = labels[i]
		seg_datas, s = segment_signal_without_transition(x_all, label, window_size)
		if len(label_all) == 0:

			label_all = s
			seg_data_all = seg_datas
		else:
			label_all = np.append(label_all, s)
			seg_data_all = np.vstack([seg_data_all, seg_datas])

print('len(label_all)', len(label_all))
print('seg_data_all', seg_data_all.shape)
D2_data = dataset_1Dto2D(seg_data_all)
print('D2_data.shape', D2_data.shape)
D3_data = np.reshape(D2_data, (len(label_all), 200*5, 10, 11))
print(D3_data.shape)

with open('./result/D3_data3_neg2.pkl', "wb") as fp:
	pickle.dump(D3_data, fp, protocol=4)
with open('./result/D3_label3_neg2.pkl', "wb") as fp:
	pickle.dump(label_all, fp)
print("over")

[ 54.65745926  62.52527237  73.31371307 ...  -1.22189522  -9.68575478
 -19.22249794]
[ 0.78924929  5.49853418 16.9802311  ... -3.33516048 -6.86257648
 -7.26679465]
[ 24.88493919  29.20627594  41.72325134 ... -13.50045204 -11.08646393
 -22.53055573]
[ 0.35933651  2.51409616  7.91355721 ... -5.19899373 -8.07789751
 -6.32932141]
[ 41.96166992  39.51787949  52.60109901 ... -25.18296242 -12.18914986
 -25.92802048]
[ 0.60592312  4.09882987 12.26953695 ... -6.84641487 -8.95603663
 -5.02201699]
[ 21.9643116   22.85838127  30.33876419 ...  -2.20537186 -16.95752144
 -21.72589302]
[ 0.31716288  2.17686269  6.64557426 ... -0.76323854 -5.37072468
 -7.35646258]
[ 31.1434269   25.06375313  35.85219383 ... -42.28949547 -20.17617226
 -10.46061516]
[ 0.44970856  2.98050071  8.70158237 ... -8.4061067  -7.07368858
 -1.67714182]
[ 39.75629807  21.45767212  28.01418304 ...  33.61701965 -36.23962402
 -22.02391624]
[  0.57407773   3.65261124   9.96568028 ...  10.11828408  -2.98720875
 -15.99391556]
[ 26.67307

[-0.07143696 -0.40778964 -0.72657685 ...  3.36224246  3.04350345
  1.005199  ]
[  -8.67247581   -5.84125519    5.7220459  ... -428.70640755 -426.26261711
 -418.6630249 ]
[-0.12522985 -0.81354094 -2.10062692 ...  0.2587098   0.58684121
  0.21096491]
[-3.06963921 -1.90734863  3.7252903  ... -5.84125519 -8.55326653
 -4.58955765]
[-0.04432534 -0.28564144 -0.70550586 ...  1.05103569  1.79407906
  1.70582006]
[ 9.35792923 11.26527786 11.59310341 ... 43.86901855 43.86901855
 46.22340202]
[0.13512774 0.94949725 2.94045637 ... 0.40145119 1.74909285 2.54651036]
[  5.27501106   5.81145287   4.61935997 ... -33.97464752 -36.38863564
 -33.25939178]
[ 0.07617073  0.52744709  1.58456091 ... -0.4163021  -0.06213595
  1.02526995]
[  3.78489494   3.09944153   3.24845314 ... -27.00090408 -30.90500832
 -27.20952034]
[ 0.05465358  0.36299485  1.04599459 ... -2.76860368 -2.75995395
 -0.81900188]
[ -4.85777855  -7.86781311  -9.03010368 ... -42.7365303  -47.62411118
 -41.24641418]
[-0.07014593 -0.52205928 -1.7

[-0.18203514 -1.14258784 -3.09527867 ...  5.86552315  4.20663152
  2.93084551]
[-5.51342964  1.37090683 -4.50015068 ... -1.4603138  -6.04987144
 -3.66568565]
[-0.07961348 -0.44378096 -1.02545541 ...  6.47947724  5.12480537
  4.53557039]
[-19.04368401 -14.21570778 -18.35823059 ...  -1.63912773  -6.34789467
  -3.12924385]
[-0.27498926 -1.8064928  -5.1760363  ...  5.83501722  6.66253189
  7.15434945]
[ -5.48362732  -0.77486038  -3.03983688 ...  -4.55975533 -11.86132431
  -6.67572021]
[-0.07918313 -0.47225984 -1.17897283 ...  5.94694114  5.23104748
  4.87894638]
[-12.18914986  -4.47034836  -8.07642937 ...  -5.63263893 -12.27855682
  -7.42077827]
[-0.17601034 -1.08943195 -2.87075657 ...  4.17960245  3.46046238
  3.39763924]
[-19.6993351   -7.59959221  -9.00030136 ...  -0.32782555  -4.41074371
  -0.68545341]
[-0.2844568  -1.76608467 -4.61254246 ...  3.63296558  1.46562912
 -0.26821599]
[-8.88109207 -1.40070915 -5.00679016 ... -0.77486038 -3.96370888
  0.95367432]
[-0.12824225 -0.76696056 -1.

[-0.11705333 -0.68502585 -1.83879611 ...  4.66236065  2.42660939
 -0.59017967]
[ -3.15904617  -2.50339508 -10.04338264 ... -16.21246338  -2.56299973
  -3.15904617]
[-0.04561637 -0.30176577 -0.97188524 ... -0.92320386 -2.91673885
 -4.51640494]
[ 2.23517418  1.63912773 -3.7252903  ... -7.1823597   2.32458115
  3.7252903 ]
[ 0.03227573  0.21160539  0.52013857 ... -3.01873171 -4.71868866
 -5.16381386]
[ 7.53998756  7.53998756  5.36441803 ... -2.17556953  2.56299973
  6.37769699]
[ 0.10887681  0.74284933  2.18258323 ... -3.16910885 -4.04808768
 -4.0231379 ]
[10.43081284  7.30156898  6.64591789 ...  3.81469727  6.16908073
 10.43081284]
[ 0.15062009  0.98247115  2.74507876 ... -3.81040283 -3.52135839
 -3.6971392 ]
[6.55651093 0.8046627  0.98347664 ... 4.82797623 5.54323196 9.00030136]
[ 0.09467549  0.56289972  1.36111842 ... -2.92022376 -3.31287476
 -3.83745797]
[ 0.         -8.64267349 -5.453825   ... 10.55002213  9.41753387
 10.22219658]
[ 0.         -0.12479951 -0.80544066 ... -2.7013431  

[-0.16998553 -1.18603546 -3.54408051 ... -3.17142461 -3.03470026
 -2.63701662]
[ 7.06315041  4.73856926 10.2519989  ... 11.20567322 11.11626625
  8.94069672]
[ 0.10199132  0.66230395  1.92457531 ... -3.96583102 -3.28717498
 -2.47994343]
[26.61347389 16.65949821 15.437603   ... 22.79877663 21.4278698
 16.57009125]
[ 0.38429641  2.47825928  6.8163037  ... -2.51792408 -3.14261916
 -3.07373682]
[ -9.08970833  -9.05990601 -12.30835915 ...   2.08616257  -1.25169754
  -2.59280205]
[-0.13125465 -0.89509948 -2.71302054 ... -3.4220206  -3.6485885
 -2.04595014]
[-25.45118332 -28.13339233 -29.38508987 ...   1.60932541   0.11920929
   0.92387199]
[-0.36751303 -2.54621439 -7.75566717 ... -1.52532616 -3.63468119
 -4.67882125]
[-21.87490463 -23.93126488 -22.5007534  ...  -2.44379044  -4.7981739
  -3.1888485 ]
[-0.31587185 -2.18483758 -6.6051591  ... -0.33264551 -3.01397748
 -4.4228555 ]
[-22.61996269 -25.45118332 -19.37150955 ...  -1.81794167  -3.87430191
  -1.37090683]
[-0.32663043 -2.26943059 -6.833

...

[ 0.10371269  0.79798743  2.83574865 ... -2.33280075 -2.25688781
 -0.70302366]
[ -5.39422035  -5.69224358  -3.09944153 ... -22.73917198 -20.98083496
 -19.46091652]
[-0.07789211 -0.535749   -1.5758494  ... -4.85620653 -4.58892511
 -3.57863615]
[-14.39452171 -17.1661377  -13.05341721 ... -20.80202103 -14.7819519
 -12.5169754 ]
[-0.20785573 -1.45818881 -4.44039982 ... -6.01294281 -6.35345122
 -5.49351477]
[ -7.39097595 -10.49041748   2.02655792 ... -37.07408905 -29.05726433
 -28.58042717]
[ -0.10672509  -0.7729242   -2.29486025 ...  -5.56931979  -9.31837262
 -11.31161603]
[  9.08970833   6.64591789   5.09619713 ... -33.22958946 -26.94129944
 -26.88169479]
[ 0.13125465  0.86024169  2.40590607 ... -0.4227017  -4.85946627
 -4.31625471]
[  1.01327896  -1.60932541  -5.27501106 ... -19.75893974 -14.39452171
 -15.73562622]
[ 0.01463167  0.06195936 -0.01378121 ...  0.28058978 -3.82171788
 -4.00458762]
[18.38803291 14.75214958 13.91768456 ... -3.12924385  0.47683716
 -0.5364418 ]
[ 0.26552171  1.7

[0.18246548 1.18210317 3.29593756 ... 4.64928055 7.37427616 6.09803923]
[29.951334   18.89467239 20.44439316 ... 34.03425217 40.85898399
 37.16349602]
[0.43249484 2.79118688 7.72779747 ... 0.98374576 4.74669503 6.86640909]
[29.26588058 19.46091652 20.62320709 ... 33.58721733 40.59076309
 36.26942635]
[0.42259695 2.74172953 7.6442493  ... 1.26979301 5.13972336 7.27934678]
[22.05371857 19.60992813 18.74566078 ... 33.52761269 40.08412361
 33.43820572]
[ 0.31845391  2.13747275  6.22248084 ...  3.51513518  1.58677737
 -0.51852032]
[22.26233482 19.13309097 19.58012581 ... 33.4084034  40.35234451
 33.19978714]
[0.32146631 2.148128   6.23514105 ... 3.58094351 1.91030973 0.05264182]
[18.26882362 17.37475395 16.74890518 ... 13.7090683  19.81854439
 13.32163811]
[0.26380033 1.78695785 5.26722945 ... 6.5228676  4.76229754 1.59784324]
[13.61966133 13.58985901 12.18914986 ...  4.23192978  8.64267349
  4.82797623]
[0.19666681 1.34139631 3.97603618 ... 5.35567483 5.75679813 2.63371859]
[19.46091652 16

[-0.16180901 -1.03858527 -2.84665421 ... -1.4769682  -0.13963589
  0.79750154]
[ 9.89437103 13.82827759 14.90116119 ... 12.06994057  4.73856926
  6.85453415]
[ 0.14287392  1.03161153  3.30839172 ... -1.99325342 -0.83988205
 -0.0398914 ]
[10.04338264 13.61966133 14.18590546 ... 13.38124275  5.7220459
  7.77840614]
[ 0.14502563  1.04112823  3.30959683 ... -2.08063528 -1.05735329
 -0.42541522]
[12.36796379 11.08646393 10.01358032 ...  2.89082527 -7.24196434
 -3.96370888]
[ 0.1785924   1.20000304  3.48990988 ...  1.55991766  0.95439094
 -1.15936533]
[ -1.43051147  -5.27501106   2.86102295 ... -12.75539398 -14.99056816
 -12.54677773]
[-0.02065647 -0.1964501  -0.6813285  ...  2.45579477  4.12099198
  4.24899473]
[17.73238182 12.18914986 17.1661377  ... -6.04987144 -8.7916851
 -6.40749931]
[0.25605416 1.66697338 4.73257481 ... 5.20719276 6.30813381 5.0184738 ]
[17.9708004  12.30835915 16.80850983 ... -5.57303429 -8.04662704
 -6.73532486]
[0.2594969  1.68874131 4.78395252 ... 5.22178334 6.4025

[ 0.19322406  1.28907402  3.76912308 ... -3.09291295 -1.60109913
 -0.07395441]
[  6.73532486   4.41074371   8.16583633 ... -22.53055573 -22.94778824
 -19.75893974]
[ 0.09725755  0.63000616  1.80292427 ... -1.41874364 -0.4097554
  0.64922616]
[ 2.17556953 -2.05636024  0.89406967 ...  1.01327896 -1.51991844
  1.57952309]
[ 0.03141505  0.1532312   0.26449041 ...  0.3940938   0.29440901
 -0.03206665]
[-12.21895218 -15.52700996 -11.20567322 ...  22.41134644  20.29538155
  23.18620682]
[-0.17644068 -1.25159506 -3.85141673 ... -1.33036764 -1.71238692
 -1.90696667]
[ -8.76188278 -12.01033592  -9.29832458 ...  27.26912498  26.13663673
  29.6831131 ]
[-0.12652088 -0.91013943 -2.85366909 ... -2.45045429 -2.10932434
 -1.82018083]
[-2.98023224e-02 -7.15255737e-01  6.85453415e-01 ...  5.23924828e+01
  5.03659248e+01  5.45680523e+01]
[-4.30343122e-04 -1.28340552e-02 -5.60566118e-02 ... -2.04829210e+00
 -1.84912485e+00 -1.89115867e+00]
[12.48717308 18.92447472 18.06020737 ... -1.57952309 -2.89082527
 

the shape of segments is: (41000, 62)
the shape of labels_one is: (41,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (46000, 62)
the shape of labels_one is: (46,)
the shape of segments is: (41000, 62)
the shape of labels_one is: (41,)
the shape of segments is: (41000, 62)
the shape of labels_one is: (41,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (46000, 62)
the shape of labels_one is: (46,)
the shape of segments is: (41000, 62)
the shape of labels_one is: (41,)
the shape of segments is: (41000, 62)
the shape of labels_one is: (41,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (47000, 62)
the shape of labels_one is: (47,)
the shape of segments is: (46000, 62)
the shape of labels_one is